In [1]:
from NICU_offline.hobo import enriched_hobo_plot, enriched_hobo_plot_2
from NICU_offline.pqm import find_outages, find_non_outages, read_pqm_csv
import glob
import numpy
import pandas
import pyarrow.feather as feather
import re

def build_catalog(catalogs):
    catalogs = [pandas.read_csv(c, header = 2) for c in catalogs]
    c = pandas.concat(catalogs)
    c = c.loc[c['Device deployed'].notnull()]
    c['Equipment Type'] = c['Equipment Type'].str.replace("^Radiant  warmer$", "Radiant Warmer")
    c['Equipment Type'] = c['Equipment Type'].str.replace("^Radiant  Warmer$", "Radiant Warmer")
    c['Equipment Type'] = c['Equipment Type'].str.replace("^Medical Oxygen concentrator$", "Oxygen Concentrator")
    c['Equipment Type'] = c['Equipment Type'].str.replace("^Bubble CPAP unit$", "Bubble CPAP Unit")
    c['Equipment Type'] = c['Equipment Type'].str.replace("^Infant Warmer\\(NEOTHERM/AVI\\)$", "Radiant Warmer")
    c['Equipment Type'] = c['Equipment Type'].str.replace("^Suction pump/Aspirators$", "Suction Pump/Aspirator")
    c['Equipment Type'] = c['Equipment Type'].str.replace("^ ?Phototherapy unit$", "Phototherapy Unit")
    c['Equipment Type'] = c['Equipment Type'].str.replace("^LED Phototherapy unit$", "Phototherapy Unit (LED Type)")
    c['Equipment Type'] = c['Equipment Type'].str.replace("^Phototherapy Unit \\(LED Type\\)$", "Phototherapy Unit (LED type)")
    return(c)

def catalog_info(dd, catalog):
    print("looking up {} in catalog".format(dd))
    et = catalog.loc[catalog['Device deployed'] == dd]['Equipment Type']
    return et.to_list()[0]

catalogs = ['NICU Equipment Catalogue - DWH,Amravati.csv',
            'NICU Equipment Catalogue - GH,Bhandara.csv',
            'NICU Equipment Catalogue - SDH,Hinganghat.csv']
thresholds = {
    "Oxygen Concentrator": 0.2, # rated at 550VA = 2.2A
    "Suction Pump/Aspirator": 0.2,  # rated at 200W = 0.8A. C-11 seems to exceed this rating considerably. This threshold misses a section worth looking at on C-20.
    "Phototherapy Unit": 0.03,  # rated at 30W, 80W, 120W -> 0.12A to 0.48A. What is up with sky-high current on C-03? It's only rated at 80W.
    "Bubble CPAP Unit":  0.03,  # no current rating in equipment catalog. No apparent usage. A random one I looked at can peak at 15VA = 0.06A
    "Phototherapy Unit (LED type)": 0.015,  # rated 30W, 50W, 110W -> 0.12A to 0.44A. C-36 has no rating but looks like it's in use at 0.1. C-14 is insane, probably just omit it.
    "Infusion Pump": 0.015 ,  # rated 10VA, 12VA -> 0.04A to 0.048A. C-31 is insane, showing 625VA, probably toss. C-12 and C-13 are good examples of why this is tough. Peak draw (when pump is running?) is above this threshold, but there is lower consumption probably related to timers, flow sensors, etc. But the readings during this perios are so low they're in the noise. FLAG FOR SPECIFIC FOLLOWUP FOR TIGHTER FEEDBACK, don't include in aggregates for now.
    "Pulse Oximeter": 0.01,  # rated 10VA = 0.04A. Very hard to set a threshold on this one as it's near the noise. Same as with infusion pump, need some followup to get the threshold better, current may be too low for us to do a great job.
    "Radiant Warmer": 0.2,  # rated 750W, 1100W -> 3A to 4.4A.
}

catalog = build_catalog(catalogs)

def save_enriched_plot(hobo, o, no, s, catalog, thresholds, eq_id):
    print("running catalog info function for {}".format(eq_id))
    eq_type = catalog_info(eq_id, catalog)
    threshold = thresholds[eq_type]
    plot_title = eq_type + " " + eq_id + " (Site " + s + ")"
    p = enriched_hobo_plot(hobo, o, no, title=plot_title, threshold_y=threshold, etype=eq_type)
    plot_file = "plots_combined/" + s + "/" + eq_id + ".html"
    p.write_html(plot_file)

def save_enriched_plot_2(hobo, o, no, s, catalog, thresholds, eq_id):
    print("running catalog info function for {}".format(eq_id))
    eq_type = catalog_info(eq_id, catalog)
    threshold = thresholds[eq_type]
    plot_title = eq_type + " " + eq_id + " (Site " + s + ")"
    p = enriched_hobo_plot_2(hobo, o, no, title=plot_title, threshold_y=threshold, etype=eq_type)
    plot_file = "plots_combined_new_style_with_snips_and_current_exclusions_2/" + s + "/" + eq_id + ".html"
    p.write_html(plot_file)
    

In [2]:
import altair as alt
import pandas as pd

df = pd.DataFrame({'local': ['2018-01-01T00:00:00'],
                   'utc': ['2018-01-01T00:00:00Z']})

alt.Chart(df).transform_calculate(
    compliant="hours(datum.local) != hours(datum.utc) ? true : false",
).mark_text(size=20, baseline='middle').encode(
    text=alt.condition('datum.compliant', alt.value('OK'), alt.value('not OK')),
    color=alt.condition('datum.compliant', alt.value('green'), alt.value('red'))
).properties(width=80, height=50)

alt.Chart(...)

In [3]:
def usage(hobo, etype, thresholds):
    threshold = thresholds[etype]
    hobo = hobo.groupby(hobo.index.to_period(freq='d'))
    in_use = hobo.apply(lambda x: pandas.Series({'minutes_above_threshold': numpy.sum(x['amps'] > threshold)}))
#         in_use = in_use.rename(columns = {'0': 'minutes_above_threshold'})
    return in_use
#     return hobo
    

In [4]:
# sites = ['01_02']
sites = ["01_02", "02_01", "04_01"]
# sites = ['02_01']
for s in sites:
    files = pandas.Series(glob.glob("data_cleaned_with_snips_and_current_exclusions/" + s + "/*.feather"))
    files = files.loc[~ files.str.contains('/power.feather$')]
    print(files)

    pqm = feather.read_feather("data_cleaned_with_snips_and_current_exclusions/" + s + "/power.feather")
    pqm = pqm.set_index('date')

    o = find_outages(pqm)
    o = o.assign(duration = o['end'] - o.index)
    outages_csvfile = "plots_combined_new_style_with_snips_and_current_exclusions_2/" + s + "/outages.csv"
    o.to_csv(outages_csvfile)

    no = find_non_outages(pqm)
    non_outages_csvfile = "plots_combined_new_style_with_snips_and_current_exclusions_2/" + s + "/non_outages.csv"
    no.to_csv(non_outages_csvfile)

    for f in files:
        eq_id = re.sub(r".*/(.*)\.feather", r"\1", f)
        hobo = feather.read_feather(f)
        hobo = hobo.set_index('date')
        etype = catalog_info(eq_id, catalog)
        if (etype != "Oxygen Concentrator" and etype != "Radiant Warmer"):
            hobo.loc[hobo['amps'] > 1.2, 'amps'] = numpy.NaN 
        if (etype == "Infusion Pump" or etype == "Pulse Oximeter"):
            hobo.loc[hobo['amps'] > 0.2, 'amps'] = numpy.NaN
        my_usage = usage(hobo, etype, thresholds)
        my_usage['equipment'] = numpy.repeat(eq_id, len(my_usage.index))
        my_usage['etype'] = numpy.repeat(etype, len(my_usage.index))
        my_usage['site'] = numpy.repeat(s, len(my_usage.index))
        usage_output = "plots_combined_new_style_with_snips_and_current_exclusions_2/" + s + "/" + eq_id + ".csv"
        my_usage.to_csv(usage_output)
        print("saved to {}".format(usage_output))
        save_enriched_plot_2(hobo, o, no, s, catalog, thresholds, eq_id)

1    data_cleaned_with_snips_and_current_exclusions...
2    data_cleaned_with_snips_and_current_exclusions...
3    data_cleaned_with_snips_and_current_exclusions...
4    data_cleaned_with_snips_and_current_exclusions...
dtype: object
date
2020-02-20 11:31:00+05:30     58°
2020-02-21 16:16:00+05:30    113°
2020-02-21 22:31:00+05:30    110°
2020-02-22 07:55:00+05:30     95°
2020-02-22 09:22:00+05:30    105°
                             ... 
2020-10-03 11:38:57+05:30    247°
2020-10-03 11:38:57+05:30    247°
2020-10-03 11:38:57+05:30    247°
2020-10-03 11:56:41+05:30    138°
2020-10-03 11:56:41+05:30    138°
Name: end, Length: 785, dtype: object
date
2020-02-20 11:31:00+05:30    0.000056
2020-02-21 16:16:00+05:30    0.000056
2020-02-21 22:31:00+05:30    0.000056
2020-02-22 07:55:00+05:30    0.000056
2020-02-22 09:22:00+05:30    0.000056
                               ...   
2020-10-03 11:38:57+05:30    0.000056
2020-10-03 11:38:57+05:30    0.000056
2020-10-03 11:38:57+05:30    0.000056
20

/code/NICU_offline/pqm.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  degree_endings[:] = numpy.repeat(1. / 50 / 360, len(degrees))
/home/dataportal/venv/lib/python3.8/site-packages/pandas/core/series.py:1042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
<ipython-input-4-551d830f8ff1>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no = find_non_outages(pqm)


saved to plots_combined_new_style_with_snips_and_current_exclusions_2/01_02/C-01.csv
running catalog info function for C-01
looking up C-01 in catalog
looking up C-03 in catalog
saved to plots_combined_new_style_with_snips_and_current_exclusions_2/01_02/C-03.csv
running catalog info function for C-03
looking up C-03 in catalog
looking up C-02 in catalog
saved to plots_combined_new_style_with_snips_and_current_exclusions_2/01_02/C-02.csv
running catalog info function for C-02
looking up C-02 in catalog
looking up C-04 in catalog
saved to plots_combined_new_style_with_snips_and_current_exclusions_2/01_02/C-04.csv
running catalog info function for C-04
looking up C-04 in catalog
0     data_cleaned_with_snips_and_current_exclusions...
1     data_cleaned_with_snips_and_current_exclusions...
2     data_cleaned_with_snips_and_current_exclusions...
3     data_cleaned_with_snips_and_current_exclusions...
4     data_cleaned_with_snips_and_current_exclusions...
5     data_cleaned_with_snips_and_c

/code/NICU_offline/pqm.py:127: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/dataportal/venv/lib/python3.8/site-packages/pandas/core/series.py:1042: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-551d830f8ff1>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



looking up C-35 in catalog
saved to plots_combined_new_style_with_snips_and_current_exclusions_2/02_01/C-35.csv
running catalog info function for C-35
looking up C-35 in catalog
looking up C-28-old in catalog
saved to plots_combined_new_style_with_snips_and_current_exclusions_2/02_01/C-28-old.csv
running catalog info function for C-28-old
looking up C-28-old in catalog
looking up C-33-old in catalog
saved to plots_combined_new_style_with_snips_and_current_exclusions_2/02_01/C-33-old.csv
running catalog info function for C-33-old
looking up C-33-old in catalog
looking up C-22 in catalog
saved to plots_combined_new_style_with_snips_and_current_exclusions_2/02_01/C-22.csv
running catalog info function for C-22
looking up C-22 in catalog
looking up C-26 in catalog
saved to plots_combined_new_style_with_snips_and_current_exclusions_2/02_01/C-26.csv
running catalog info function for C-26
looking up C-26 in catalog
looking up C-31 in catalog
saved to plots_combined_new_style_with_snips_and_cu

In [5]:
non_outages = pqm[pqm['description'] != "Outage"]
non_outages = non_outages.replace(to_replace="\d+ [HNG]-[HNG] Impulses?",
        value="Impulse", regex=True)
non_outages = non_outages.replace(to_replace="[HNG]-[HNG] Sag",
        value="Sag", regex=True)
non_outages = non_outages.replace(to_replace="[HNG]-[HNG] Surge",
        value="Surge", regex=True)
non_outages.columns = ['event_number', 'description', 'extreme', 'end']
#non_outages = non_outages.loc[~ non_outages['end'].str.contains('Open Event')]


~ non_outages['end'].str.contains("Open Event").astype('boolean')

date
2020-02-26 11:46:24+05:30    True
2020-02-26 11:46:24+05:30    True
2020-02-26 11:46:32+05:30    True
2020-02-26 11:46:32+05:30    True
2020-02-26 11:46:32+05:30    True
                             ... 
2020-07-06 06:47:00+05:30    True
2020-07-06 06:47:00+05:30    True
2020-07-06 12:57:00+05:30    True
2020-07-06 12:57:00+05:30    True
2020-07-06 18:57:00+05:30    True
Name: end, Length: 4203, dtype: boolean

In [6]:
hobo.tail()

,amps
date,
2020-03-28 08:47:00,NaN
2020-03-28 08:48:00,NaN
2020-03-28 08:49:00,NaN
2020-03-28 08:50:00,NaN
NaT,NaN
